In [1]:
import requests
import geopandas as gpd
import pandas as pd
import os
import json

### Extração

In [2]:
def fetch_geojson_data(
    url: str = "https://sigel.aneel.gov.br/arcgis/rest/services/PORTAL/WFS/MapServer/0/query",
    batch_size: int = 1000,
    timeout: int = 30,
) -> [dict[str,any]]:
    """
    Faz uma requisição à API e retorna os dados no formato GeoJSON.

    Parâmetros:
        
    Retorno:
        dict com os dados ou None (caso erro)
    """
    all_features = []
    offset = 0

    while True:
        params = {
            "where": "1=1",
            "outFields": "*",
            "f": "geojson",
            "resultOffset": offset,
            "resultRecordCount": batch_size
        }

        try:
            print(f" Coletando registros a partir do offset {offset}...")
            response = requests.get(url, params=params, timeout=timeout)
            response.raise_for_status()
            data = response.json()
            features = data.get("features", [])

            if not features:
                print(" Coleta concluída.")
                break

            all_features.extend(features)
            offset += batch_size

        except requests.exceptions.RequestException as e:
            print(f" Erro na página {offset}: {e}")
            break

    print(f" Total de registros coletados: {len(all_features)}")
    return {"type": "FeatureCollection", "features": all_features} if all_features else None


In [3]:
# Realiza a consulta
df = fetch_geojson_data()

 Coletando registros a partir do offset 0...
 Coletando registros a partir do offset 1000...
 Coletando registros a partir do offset 2000...
 Coletando registros a partir do offset 3000...
 Coletando registros a partir do offset 4000...
 Coletando registros a partir do offset 5000...
 Coletando registros a partir do offset 6000...
 Coletando registros a partir do offset 7000...
 Coletando registros a partir do offset 8000...
 Coletando registros a partir do offset 9000...
 Coletando registros a partir do offset 10000...
 Coletando registros a partir do offset 11000...
 Coletando registros a partir do offset 12000...
 Coletando registros a partir do offset 13000...
 Coletando registros a partir do offset 14000...
 Coletando registros a partir do offset 15000...
 Coletando registros a partir do offset 16000...
 Coletando registros a partir do offset 17000...
 Coletando registros a partir do offset 18000...
 Coletando registros a partir do offset 19000...
 Coletando registros a partir do 

In [4]:
# Salve arquivo raw
os.makedirs("../data/raw", exist_ok=True)
if df:
    with open("../data/raw/aerogeradores.geojson", "w", encoding="utf-8") as f:
        json.dump(df, f, ensure_ascii=False, indent=2)

### Processamento

In [7]:
def load_geojson(filepath: str) -> [gpd.GeoDataFrame]:
    """
    Carrega um arquivo GeoJSON e retorna um GeoDataFrame.
    """
    try:
        gdf = gpd.read_file(filepath)
        print(f" GeoDataFrame carregado com {len(gdf)} registros.")
        return gdf
    except Exception as e:
        print(f" Erro ao carregar o GeoJSON: {e}")
        return None
def add_lat_lon(gdf: gpd.GeoDataFrame) -> gpd.GeoDataFrame:
    """
    Extrai latitude e longitude da geometria e adiciona como novas colunas.
    """
    gdf = gdf.to_crs(epsg=4326)

    gdf["LATITUDE"] = gdf.geometry.y
    gdf["LONGITUDE"] = gdf.geometry.x
    return gdf

def validate_clean(gdf: gpd.GeoDataFrame) -> gpd.GeoDataFrame:
    """
    Remove duplicatas e linhas sem geometria.
    """
    before = len(gdf)
    gdf = gdf.drop_duplicates()
    gdf = gdf[gdf.geometry.notnull()]
    after = len(gdf)
    print(f" Registros após limpeza: {after} (removidos {before - after})")
    return gdf
    
def export_to_csv(gdf: gpd.GeoDataFrame, output_path: str):
    """
    Exporta o GeoDataFrame para um arquivo CSV simples (sem geometria).
    """
    try:
        df = gdf.drop(columns=["X","Y","VERSAO","geometry"])  
        df.to_csv(output_path, index=False)
        print(f" Arquivo salvo em: {output_path}")
    except Exception as e:
        print(f" Erro ao salvar CSV: {e}")


In [6]:
# Carrega arquivo Geojson

df = load_geojson("../data/raw/aerogeradores.geojson")

 GeoDataFrame carregado com 23522 registros.


In [7]:
# adiciona latitude e longitude

df = add_lat_lon(df);

In [8]:
# limpeza

df = validate_clean(df)

 Registros após limpeza: 23522 (removidos 0)


In [8]:
path_output = "../data/processed/outputs.csv"
os.makedirs("../data/processed", exist_ok=True)
export_to_csv(df,path_output)


 Arquivo salvo em: ../data/processed/outputs.csv


In [9]:
df_1 = pd.read_csv(path_output)

In [11]:
import geopandas as gpd
import pandas as pd


gdf = df[df["UF"]=="BA"]  
print(gdf.describe())
gdf['ALT_TOTAL'] = pd.to_numeric(gdf['ALT_TOTAL'], errors='coerce')
gdf['POT_MW'] = pd.to_numeric(gdf['POT_MW'], errors='coerce')

# Remove valores nulos
gdf_clean = gdf.dropna(subset=['ALT_TOTAL', 'POT_MW'])

# Calculo do coeficiente de correlação de Pearson
correlation = gdf_clean['ALT_TOTAL'].corr(gdf_clean['POT_MW'])

print(f"Coeficiente de correlação entre altitude total e potência instalada: {correlation:.3f}")

            POT_MW    ALT_TOTAL    ALT_TORRE   DIAM_ROTOR  DATA_ATUALIZACAO  \
count  8184.000000  8184.000000  8180.000000  8184.000000      8.184000e+03   
mean      4.116552   176.716371   106.211650   141.040640      1.675018e+12   
std       1.791423    28.221288    15.443362    28.990574      2.431905e+10   
min       0.004200   121.250000    75.000000    82.000000      1.609178e+12   
25%       2.350000   150.000000    90.000000   116.000000      1.666626e+12   
50%       4.200000   195.000000   115.000000   150.000000      1.666626e+12   
75%       6.000000   200.000000   120.000000   170.000000      1.693932e+12   
max       6.600000   205.000000   126.000000   170.000000      1.746463e+12   

       EOL_VERSAO_ID              X             Y      OBJECTID  
count    8184.000000    8184.000000  8.184000e+03   8184.000000  
mean    46529.248045  470667.532768  8.709460e+06  12811.963954  
std     13762.783125  243893.406745  2.632392e+05   6703.451447  
min     30283.000000    

C:\Users\Gustavo\AppData\Local\Programs\Python\Python313\Lib\site-packages\geopandas\geodataframe.py:1968: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  super().__setitem__(key, value)
